In [1]:
import sys
sys.path.append('../src/')

import config
import logging
import scanpy as sc
from refcm import RefCM

config.start_logging(logging.DEBUG)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
ds = ['MTG', 'ALM', 'VISp']
ds = {s: sc.read_h5ad(f'../data/{s}.h5ad') for s in ds}

[h5py._conv      ] [DEBUG   ] : Creating converter from 3 to 5


In [8]:
q, ref = 'MTG', 'ALM'

rcm = RefCM(cache_load=False, cache_save=False, discovery_threshold=None)
m = rcm.annotate(ds[q], q, ds[ref], ref, 'labels34', 'labels34')
m.eval('labels34')
m.display_matching_costs('labels34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
[refcm           ] [DEBUG   ] : Loading cached mapping costs from cache.json.
[embeddings      ] [DEBUG   ] : Using 1948 genes.
[refcm           ] [DEBUG   ] : Computing Wasserstein distances.
|████████████████| [100.00% ] : 00:31
[refcm           ] [DEBUG   ] : starting LP optimization
[refcm           ] [DEBUG   ] : optimization terminated w. status "Optimal"
[matchings       ] [DEBUG   ] : [+] Astrocyte            mapped to Astrocyte           
[matchings       ] [DEBUG   ] : [+] Chandelier           mapped to Chandelier          
[matchings       ] [DEBUG   ] : [+] Exc L2/3 IT          mapped to Exc L2/3 IT         
[matchings       ] [DEBUG   ] : [+] Exc L3/5 IT          mapped to Exc L3/5 IT         
[matchings       ] [DEBUG   ] : [+] Exc L4/5 IT          mapped to Exc L4/5 IT         
[matchings       ] [DEBUG   ] : [+] Exc L5 PT            mapped to Exc L5 PT           
[matchings       ] [DEBU

In [5]:
ds[q].obs

,labels3,labels34,refcm_clusters,refcm_annot
F1S4_160106_001_B01,Inhibitory,Sst 1,22,Sst 1
F1S4_160106_001_C01,Excitatory,Exc L5/6 IT 3,8,Exc L5/6 IT 3
F1S4_160106_001_E01,Excitatory,Exc L6 CT,10,Exc L6 CT
F1S4_160106_001_G01,Excitatory,Exc L6b,13,Exc L6b
F1S4_160106_001_H01,Excitatory,Exc L4/5 IT,4,Exc L4/5 IT
...,...,...,...,...
F2S4_170405_060_A01,Excitatory,Exc L6 IT 1,11,Exc L6 IT 1
F2S4_170405_060_B01,Excitatory,Exc L6 CT,10,Exc L6 CT
F2S4_170405_060_C01,Excitatory,Exc L6 IT 2,12,Exc L6 IT 2
F2S4_170405_060_E01,Excitatory,Exc L5/6 NP,9,Exc L5/6 NP


In [23]:
x_qc = ds[q][ds[q].obs['labels34'] == 'Sst 1'].X
x_rc = ds[ref][ds[ref].obs['labels34'] == 'Sst 1'].X

In [73]:
pairwise_distances(x_qc, x_rc, 'minkowski')

array([[59909.19050743, 56357.54638598, 52861.26636036, ...,
        61214.78589675, 62135.52360843, 62075.6845127 ],
       [67211.92684129, 63463.76467347, 60927.44072739, ...,
        67983.56214984, 68538.19934368, 68490.28972909],
       [90120.8554905 , 85935.18254654, 82238.57715996, ...,
        91429.04553188, 92395.87295139, 92111.88221149],
       ...,
       [53138.74682112, 51214.56015132, 47497.87477183, ...,
        56756.65536079, 57870.05302754, 57168.47178404],
       [62435.16385963, 60938.99204681, 57158.60263075, ...,
        65852.23984812, 66471.62188793, 65906.88750941],
       [87827.94473116, 83447.58410701, 79276.44160075, ...,
        87818.80049358, 88217.90147435, 88358.94445056]])

In [70]:
(x_qc @ x_rc.T) / (norm(x_qc, axis=1).reshape(-1, 1) @ norm(x_rc, axis=1).reshape(1, -1))

array([[0.27448392, 0.4381256 , 0.5240184 , ..., 0.32948738, 0.31564957,
        0.30325562],
       [0.2888411 , 0.4324723 , 0.495603  , ..., 0.3288856 , 0.32230005,
        0.31314394],
       [0.13322608, 0.31570604, 0.412918  , ..., 0.18174323, 0.16464649,
        0.16035235],
       ...,
       [0.2333373 , 0.39687553, 0.49045992, ..., 0.27253112, 0.2565909 ,
        0.2553947 ],
       [0.18037313, 0.324394  , 0.42278203, ..., 0.21361275, 0.20980869,
        0.20703687],
       [0.22946192, 0.3800401 , 0.48740515, ..., 0.273969  , 0.269046  ,
        0.25815186]], dtype=float32)

In [33]:
q, ref = 'MTG', 'ALM'

rcm = RefCM(cache_load=False, cache_save=False, discovery_threshold=None)
m = rcm.annotate(ds[q], q, ds[ref], ref, 'labels34', 'labels34')
m.eval('labels34')
m.display_matching_costs('labels34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
[refcm           ] [DEBUG   ] : Loading cached mapping costs from cache.json.


In [6]:
rcm = RefCM(discovery_threshold=0)

for i, q_id in enumerate(ds):
    for ref_id in [d for d in ds if d != q_id]:
        m = rcm.annotate(ds[q_id], q_id, ds[ref_id], ref_id, 'labels34', 'labels34')
        m.eval('labels34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
|████████████████| [100.00% ] : 00:38
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 00:43
[matchings       ] [INFO    ] : mapped MTG                  to VISp                
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 00:36
[matchings       ] [INFO    ] : mapped ALM                  to MTG                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 01:01
[matchings 